# Superfast Supertrend Multiple Parameters

VectorBT provides a robust framework for optimizing trading strategies more efficiently, enabling the selection of the best parameters and fine-tuning of the strategy. 

In this tutorial, we will explore the process of building a Supertrend using the Talib, Numpy and Numba Python libraries. This approach aims to reduce computation time and enhance the calculation speed of the Supertrend indicator, which is crucial for optimizing strategies with larger parameter datasets.

reference: https://www.marketcalls.in/python/mastering-vectorbt-superfast-supertrend-grid-optimization-part-3-python-tutorial.html

more detailed version using vectorbt pro: https://medium.datadriveninvestor.com/superfast-supertrend-6269a3af0c2a

## Imports

In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from numba import njit
import talib

## Parameters

In [14]:
# Define params
InitialCapital = 2000000

# Parameter Grid and Compute Supertrend for Multiple Combinations
length   = np.arange(10,100,1)
multiplier = np.arange(1,5,0.25)

## Data

In [2]:
# Get data
data = pd.read_pickle("data/yfinance_mag7.pkl")
data.head()

Price           Close                                                          \
Ticker           AAPL       AMZN      GOOGL        META        MSFT      NVDA   
Date                                                                            
2020-03-13  67.457481  89.250000  60.424614  169.632019  152.028717  5.998899   
2020-03-16  58.779289  84.457497  53.394722  145.454376  129.621109  4.891977   
2020-03-17  61.363819  90.391998  55.637001  148.851395  140.293671  5.411811   
2020-03-18  59.861629  91.500000  54.299900  146.400772  134.387878  5.051887   
2020-03-19  59.402973  94.046501  55.319023  152.547287  136.598984  5.304707   

Price                 Dividends             ... Stock Splits            \
Ticker           TSLA      AAPL AMZN GOOGL  ...         MSFT NVDA TSLA   
Date                                        ...                          
2020-03-13  36.441334       0.0  0.0   0.0  ...          0.0  0.0  0.0   
2020-03-16  29.671333       0.0  0.0   0.0  ...          0.0  0.0  0.0   
2020-03-17  28.680000       0.0  0.0   0.0  ...          0.0  0.0  0.0   
2020-03-18  24.081333       0.0  0.0   0.0  ...          0.0  0.0  0.0   
2020-03-19  28.509333       0.0  0.0   0.0  ...          0.0  0.0  0.0   

Price          Volume                                                      \
Ticker           AAPL       AMZN     GOOGL      META      MSFT       NVDA   
Date                                                                        
2020-03-13  370732000  176194000  79400000  35028600  92727400  634836000   
2020-03-16  322423600  178346000  96520000  39120400  87905900  726972000   
2020-03-17  324056000  218342000  83194000  34255600  81059800  833632000   
2020-03-18  300233600  192904000  93044000  37553100  81593200  874268000   
2020-03-19  271857200  207998000  74064000  39862300  85922700  765512000   

Price                  
Ticker           TSLA  
Date                   
2020-03-13  339604500  
2020-03-16  307342500  
2020-03-17  359919000  
2020-03-18  356793000  
2020-03-19  452932500  

[5 rows x 49 columns]

In [5]:
data = data.loc[:,pd.IndexSlice[:, 'TSLA']].droplevel(1, axis=1)
data.head()


## Functions

In [8]:
# Function definitions
def get_basic_bands(med_price, atr, multiplier):
    matr = multiplier * atr
    upper = med_price + matr
    lower = med_price - matr
    return upper, lower


@njit
def get_final_bands_nb(close, upper, lower):
    trend = np.full(close.shape, np.nan)
    dir_ = np.full(close.shape, 1)
    long = np.full(close.shape, np.nan)
    short = np.full(close.shape, np.nan)

    for i in range(1, close.shape[0]):
        if close[i] > upper[i - 1]:
            dir_[i] = 1
        elif close[i] < lower[i - 1]:
            dir_[i] = -1
        else:
            dir_[i] = dir_[i - 1]
            if dir_[i] > 0 and lower[i] < lower[i - 1]:
                lower[i] = lower[i - 1]
            if dir_[i] < 0 and upper[i] > upper[i - 1]:
                upper[i] = upper[i - 1]

        if dir_[i] > 0:
            trend[i] = long[i] = lower[i]
        else:
            trend[i] = short[i] = upper[i]

    return trend, dir_, long, short

def faster_supertrend_talib(high, low, close, period=7, multiplier=3):
    avg_price = talib.MEDPRICE(high, low)
    atr = talib.ATR(high, low, close, period)
    upper, lower = get_basic_bands(avg_price, atr, multiplier)
    return get_final_bands_nb(close, upper, lower)

# Initialize IndicatorFactory to create a basic structure or template for a supertrend indicator and then use a class method such as IndicatorFactory.from_apply_func() to bind a supertrend calculation function to the template
SuperTrend = vbt.IndicatorFactory(
    class_name='SuperTrend',
    short_name='st',
    input_names=['high', 'low', 'close'],
    param_names=['period', 'multiplier'],
    output_names=['supert', 'superd', 'superl', 'supers']
).from_apply_func(
    faster_supertrend_talib,
    period=7,
    multiplier=3,
    to_2d = False
)


## Backtest

In [15]:
# Compute Supertrend for Multiple Combinations
st = SuperTrend.run(data['High'], data['Low'], data['Close'],
                    period=length,multiplier=multiplier,param_product = True)

#Trading Logic - while backtesting vectorbt autmatically removes excessive signals
entries = st.superd==1
exits = st.superd==-1

In [16]:
# Run the Backtesting Engine for Multiple Combinational Parameters and Get Porfolio Stats with Selected Metrics
portfolio = vbt.Portfolio.from_signals(
    data['Close'],
    entries=entries,
    exits=exits,
    size = 50,
    size_type = 'percent',
    fees = 0.001,
    init_cash = InitialCapital,
    min_size = 1,
    size_granularity = 1,
    freq = '1D'
)

#get portfolio stats
stats = portfolio.stats([
    'total_return',
    'total_trades',
    'win_rate',
    'expectancy', 
    'profit_factor',
    'sharpe_ratio',
    'calmar_ratio',
    'sortino_ratio'
], agg_func=None)
 
stats

Total Return [%]  Total Trades  Win Rate [%]  \
st_period st_multiplier                                                 
10        1.00                 383.925832            74     39.189189   
          1.25                 380.128547            60     40.000000   
          1.50                 439.529611            46     39.130435   
          1.75                 962.512437            35     48.571429   
          2.00                 852.558296            31     54.838710   
...                                   ...           ...           ...   
99        3.75                 826.019447            15     46.666667   
          4.00                 750.406197            15     46.666667   
          4.25                 321.642629            15     40.000000   
          4.50                 267.378043            15     40.000000   
          4.75                 488.072957            13     46.153846   

                           Expectancy  Profit Factor  Sharpe Ratio  \
st_period st_multiplier                                              
10        1.00           1.037637e+05       1.441989      1.092378   
          1.25           1.267095e+05       1.448315      1.092170   
          1.50           1.910998e+05       1.607718      1.135111   
          1.75           5.500071e+05       2.192053      1.481772   
          2.00           5.500376e+05       2.293891      1.433100   
...                               ...            ...           ...   
99        3.75           1.101359e+06       2.242793      1.394336   
          4.00           1.000542e+06       2.159170      1.348673   
          4.25           4.288568e+05       1.520167      1.014876   
          4.50           3.565041e+05       1.425201      0.944192   
          4.75           7.508815e+05       1.823865      1.165462   

                         Calmar Ratio  Sortino Ratio  
st_period st_multiplier                               
10        1.00               1.210914       1.703780  
          1.25               1.141264       1.716938  
          1.50               1.070727       1.783647  
          1.75               2.483255       2.373479  
          2.00               2.289060       2.284419  
...                               ...            ...  
99        3.75               1.640668       2.190713  
          4.00               1.526312       2.118049  
          4.25               0.745058       1.561521  
          4.50               0.628918       1.449074  
          4.75               1.041531       1.833320  

[1440 rows x 8 columns]

### Calmar Ratio max

In [17]:
# Sort the Dataframe with Highest Calmar Ratio
df = stats.sort_values("Calmar Ratio", ascending=False)
print('Sorting by Calmar Ratio')
df

Sorting by Calmar Ratio


,,Total Return [%],Total Trades,Win Rate [%],Expectancy,Profit Factor,Sharpe Ratio,Calmar Ratio,Sortino Ratio
st_period,st_multiplier,,,,,,,,
10,2.75,1452.473271,17,52.941176,1.708792e+06,3.962722,1.706098,3.308258,2.820833
42,1.00,1222.729796,73,40.277778,3.400146e+05,1.676088,1.611464,3.295987,2.646566
43,1.00,1222.729796,73,40.277778,3.400146e+05,1.676088,1.611464,3.295987,2.646566
10,2.50,1216.459824,21,57.142857,1.158533e+06,2.938550,1.606507,3.287371,2.641583
46,1.00,1187.276222,73,40.277778,3.301565e+05,1.672245,1.597714,3.246879,2.623235
...,...,...,...,...,...,...,...,...,...
82,4.50,177.631988,16,43.750000,2.220400e+05,1.379658,0.806132,0.474762,1.220309
83,4.50,177.631988,16,43.750000,2.220400e+05,1.379658,0.806132,0.474762,1.220309
81,4.50,177.631988,16,43.750000,2.220400e+05,1.379658,0.806132,0.474762,1.220309


In [19]:
# Get the Best Parameters with Max Calamar Ratio
max_calmar_index = df['Calmar Ratio'].idxmax()

# Get the corresponding values for st_period and st_multiplier
max_calmar_values = df.loc[max_calmar_index]

st_best_period, st_best_multiplier = max_calmar_index

print('Best Parameters for Calmar Ratio')
print("Best st_period:", st_best_period)
print("Best st_multiplier:", st_best_multiplier)

print("Max Calmar Ratio Row:")
print(max_calmar_values)

Best Parameters for Calmar Ratio
Best st_period: 10
Best st_multiplier: 2.75
Max Calmar Ratio Row:
Total Return [%]    1.452473e+03
Total Trades        1.700000e+01
Win Rate [%]        5.294118e+01
Expectancy          1.708792e+06
Profit Factor       3.962722e+00
Sharpe Ratio        1.706098e+00
Calmar Ratio        3.308258e+00
Sortino Ratio       2.820833e+00
Name: (10, 2.75), dtype: float64


In [21]:
# Get the portfolio Backtesting Stats for the Best Parameters
portfolio[(st_best_period,st_best_multiplier)].stats()

Start                         2020-03-13 00:00:00
End                           2025-03-12 00:00:00
Period                         1256 days 00:00:00
Start Value                             2000000.0
End Value                         31049465.417728
Total Return [%]                      1452.473271
Benchmark Return [%]                   580.792854
Max Gross Exposure [%]                  99.999936
Total Fees Paid                     478366.870927
Max Drawdown [%]                        36.841064
Max Drawdown Duration           314 days 00:00:00
Total Trades                                   17
Total Closed Trades                            17
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            52.941176
Best Trade [%]                           271.8156
Worst Trade [%]                        -21.614022
Avg Winning Trade [%]                   59.290142
Avg Losing Trade [%]                    -8.458155


In [22]:
#Get the trade list of best parameters
trades = portfolio[(st_best_period,st_best_multiplier)].trades.records_readable
trades

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,"(10, 2.75)",54827.0,2020-03-13,36.441334,1997.969007,2020-09-03,135.666672,7438.196612,5.430791e+06,2.718156,Long,Closed,0
1,1,"(10, 2.75)",45762.0,2020-11-18,162.213333,7423.206551,2021-02-18,262.459991,12010.694129,4.568054e+06,0.615375,Long,Closed,1
2,2,"(10, 2.75)",47172.0,2021-04-13,254.106674,11986.720035,2021-05-10,209.679993,9891.024615,-2.117573e+06,-0.176660,Long,Closed,2
3,3,"(10, 2.75)",45104.0,2021-06-23,218.856674,9871.311433,2021-08-17,221.903336,10008.728048,1.175366e+05,0.011907,Long,Closed,3
4,4,"(10, 2.75)",40998.0,2021-08-30,243.636673,9988.616319,2021-11-09,341.166656,13987.150583,3.974558e+06,0.397909,Long,Closed,4
5,5,"(10, 2.75)",39248.0,2021-12-23,355.666656,13959.204934,2022-01-19,331.883331,13025.756987,-9.604329e+05,-0.068803,Long,Closed,5
6,6,"(10, 2.75)",42337.0,2022-03-21,307.053345,12999.717456,2022-04-11,325.309998,13772.649367,7.461595e+05,0.057398,Long,Closed,6
7,7,"(10, 2.75)",50588.0,2022-07-21,271.706665,13745.096771,2022-09-02,270.209991,13669.383048,-1.031282e+05,-0.007503,Long,Closed,7
8,8,"(10, 2.75)",44814.0,2022-09-12,304.420013,13642.278482,2022-09-23,275.329987,12338.638018,-1.329621e+06,-0.097463,Long,Closed,8
9,9,"(10, 2.75)",85662.0,2023-01-23,143.750000,12313.912500,2023-03-09,172.919998,14812.672883,2.471634e+06,0.200719,Long,Closed,9


### Heatmaps

By using a heatmap, you can compare the Sharpe Ratios of various parameter combinations at a glance. This makes it easier to identify regions of parameter space that lead to better trading performance. It aids in making informed decisions about which parameter values are most suitable for your specific trading goals.

Heatmaps allow you to see how sensitive the Sharpe Ratio is to changes in ATR Length and Multiplier. Understanding the sensitivity helps you assess the robustness of the strategy. For example, you might identify regions where the strategy performs well and is less sensitive to parameter changes, indicating a more stable configuration.

In [23]:
# Get the Sharpe Ratio Heatmap against Supertrend Parameter Combinations
import plotly.graph_objects as go

sharpe = portfolio.sharpe_ratio()
calmar = portfolio.calmar_ratio()

# Plot Sharpe Ratio heatmap
sharpe_fig = sharpe.vbt.heatmap(
    x_level="st_period",
    y_level="st_multiplier",
)

sharpe_fig.update_layout(
    title='Sharpe Ratio Heatmap',
    xaxis_title="st_length",
    yaxis_title="st_multiplier",
    width=800,  # set the width of the figure
    height=600   # set the height of the figure
)

sharpe_fig.show()

In [24]:
# Plot Calmar Ratio heatmap
calmar_fig = calmar.vbt.heatmap(
    x_level="st_period",
    y_level="st_multiplier",
)

calmar_fig.update_layout(
    title='Calmar Ratio Heatmap',
    xaxis_title="st_length",
    yaxis_title="st_multiplier",
    width=800,  # set the width of the figure
    height=600   # set the height of the figure
)

calmar_fig.show()